In [1]:
pwd

'/Users/quanghuypham'

In [2]:
cd /Users/quanghuypham/projects

/Users/quanghuypham/projects


In [3]:
pwd

'/Users/quanghuypham/projects'

In [4]:
from map_gen import *

In [5]:
import os
from PIL import Image
from geopy.geocoders import Nominatim
import folium
from folium.plugins import MarkerCluster
from GPSPhoto import gpsphoto
from tqdm import tqdm
import csv
from math import radians, sin, cos, sqrt, atan2
import time
from collections import defaultdict
from sklearn.cluster import DBSCAN
import numpy as np
from qdrant_client import QdrantClient, models
from config import qdrant_config

In [6]:
qdrant_client = QdrantClient(
    qdrant_config.url,
    prefer_grpc=True,
    api_key= qdrant_config.api_key,
)

In [7]:
qdrant_client.get_collection(collection_name=qdrant_config.collection_name)

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=5703, indexed_vectors_count=0, points_count=5703, segments_count=2, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=256, distance=<Distance.DOT: 'Dot'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), payload_schema={})

In [8]:
all_result = data_scroll(qdrant_client)

In [9]:
all_result

[Record(id='009d03c5-8161-478e-a45c-c335dcd4578f', payload={'filetype': 'image', 'timestamp': -1, 'url': 'https://storage.googleapis.com/glimpse-compai.appspot.com/images/847.JPG', 'locations': {}}, vector=None),
 Record(id='00fa4ed6-996b-4777-9649-51e01ed72d3a', payload={'locations': {}, 'url': 'https://storage.googleapis.com/glimpse-compai.appspot.com/videos/857.MP4', 'fps': 30.0, 'filetype': 'video', 'keyframes_url': ['https://storage.googleapis.com/glimpse-compai.appspot.com/keyframes/857_6.jpg', 'https://storage.googleapis.com/glimpse-compai.appspot.com/keyframes/857_13.jpg', 'https://storage.googleapis.com/glimpse-compai.appspot.com/keyframes/857_20.jpg', 'https://storage.googleapis.com/glimpse-compai.appspot.com/keyframes/857_33.jpg', 'https://storage.googleapis.com/glimpse-compai.appspot.com/keyframes/857_41.jpg', 'https://storage.googleapis.com/glimpse-compai.appspot.com/keyframes/857_52.jpg'], 'keyframes_uuid': ['f65dbecf-f830-4cbb-a290-2af8a87ceb22', 'eb3fdebd-daaf-466c-9c5d

In [10]:
all_result[0]

Record(id='009d03c5-8161-478e-a45c-c335dcd4578f', payload={'filetype': 'image', 'timestamp': -1, 'url': 'https://storage.googleapis.com/glimpse-compai.appspot.com/images/847.JPG', 'locations': {}}, vector=None)

In [11]:
cnt = qdrant_client.count(
        collection_name=qdrant_config.collection_name, 
        
        count_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="filetype",
                    match=models.MatchAny(
                        any=['image', 'video'])
                ),
            ]
        ),
)
len(all_result)
cnt

CountResult(count=861)

In [12]:
# geolocator = Nominatim(user_agent="photo_location_extractor") #get_location_name
# image_locations = []
# t3 =time.time()
image_locations = save_image_locations_without_locname(all_result)
# t4=time.time()
# print(f'IMPORT TIME: {t4-t3} s')

In [13]:
image_locations

[{'id': '0113e939-d8cc-4cdb-b6a1-603713a98530',
  'latitude': 10.7789,
  'longitude': 106.69719,
  'image_path': 'https://storage.googleapis.com/glimpse-compai.appspot.com/videos/796.MP4'},
 {'id': '0115c98c-c7de-47be-8fcc-0dfbb78e47fa',
  'latitude': 16.01518888888889,
  'longitude': 108.25097777777778,
  'image_path': 'https://storage.googleapis.com/glimpse-compai.appspot.com/images/224.jpeg'},
 {'id': '019a1675-ded7-49d2-a696-6d5012e9e4d1',
  'latitude': 16.451805555555556,
  'longitude': 107.48921111111112,
  'image_path': 'https://storage.googleapis.com/glimpse-compai.appspot.com/images/582.JPEG'},
 {'id': '01a37c27-22df-4c74-a75f-70b6651b426a',
  'latitude': 15.967316666666669,
  'longitude': 108.260925,
  'image_path': 'https://storage.googleapis.com/glimpse-compai.appspot.com/images/544.JPEG'},
 {'id': '0223b207-c546-493e-8470-2f400e31c808',
  'latitude': 15.9758,
  'longitude': 108.2614,
  'image_path': 'https://storage.googleapis.com/glimpse-compai.appspot.com/videos/547.MP4'

In [14]:
len(image_locations)

589

In [15]:
coordinates, labels = cluster_db(image_locations)

cluster succesfully!


In [16]:
map = folium.Map(location=[image_locations[0]['latitude'], image_locations[0]['longitude']], zoom_start=9)
cluster_and_visualize(map, image_locations)
map #remove

In [17]:
centroids = cluster_centroids(coordinates, labels)
centroids

{0: array([ 10.92871509, 106.82993083]),
 1: array([ 16.13171458, 108.08227997]),
 2: array([ 18.76400604, 105.72184104]),
 3: array([ 11.87800354, 108.41593299]),
 4: array([ 10.93844081, 108.23452927]),
 5: array([ 13.87355976, 108.14038517])}

In [18]:
print_centroids(centroids)

Centroid 1 'Phường Hiệp Hòa, Thành phố Biên Hòa, Tỉnh Đồng Nai, 76107, Việt Nam': 
[Latitude=10.928715092592592 | Longitude=106.82993083333332]

Centroid 2 'Steeper Central Path to Climb Up, Cầu Khe Răm, Xã Hòa Bắc, Huyện Hòa Vang, Thành phố Đà Nẵng, Việt Nam': 
[Latitude=16.13171457896534 | Longitude=108.08227996771778]

Centroid 3 'Nghi Lộc, Tỉnh Nghệ An, Việt Nam': 
[Latitude=18.764006041666676 | Longitude=105.72184104166668]

Centroid 4 'Hoa Hồng, Thành phố Đà Lạt, Tỉnh Lâm Đồng, 66116, Việt Nam': 
[Latitude=11.878003541666668 | Longitude=108.4159329861111]

Centroid 5 'Tỉnh Bình Thuận, Việt Nam': 
[Latitude=10.93844081196581 | Longitude=108.23452927350426]

Centroid 6 'Đắk Đoa, Tỉnh Gia Lai, Việt Nam': 
[Latitude=13.873559763888888 | Longitude=108.14038517361112]



In [19]:
create_centroid_markers(map, centroids)
map #remove

In [20]:
display_map(map, centroids)

----------------------------------------------------------------------
INPUT PLACE: 

 phan thiết



 => Latitude: 10.931025 | Longitude: 108.1014632

 FOUND NEAREST LOCATION: Tỉnh Bình Thuận, Việt Nam | 
 MINIMUM DISTANCE: 14.55 kms | LATITUDE: 10.93844081196581 | LONGITUDE: 108.23452927350426

 RE-ROUTING TO : Tỉnh Bình Thuận, Việt Nam


In [21]:
cd /Users/quanghuypham/projects/map

/Users/quanghuypham/projects/map


In [22]:
map.save('united_map_render.html')

In [23]:
uuid = '0115c98c-c7de-47be-8fcc-0dfbb78e47fa'
show_single_map(uuid, all_result)